In [68]:
import glob
import pandas as pd
import os
import numpy as np

## Location of the Raw Data from PsychoPy

In [2]:
raw_data_loc='data_raw'

## CSV Templates for RedCap Upload Formatting

In [26]:
cfmt_u_upload=pd.read_csv('upload_templates/cfmt_u_template.csv')
rmie_upload=pd.read_csv('upload_templates/rmie_template.csv')
cfmt_i_upload=pd.read_csv('upload_templates/cfmt_i_template.csv')

## CFMT Upright 

In [3]:
cfmt_u_files=glob.glob(raw_data_loc  +  "/*CFMT_U*.csv")

In [27]:
for i in cfmt_u_files:

    subj_id=i.split("/")[-1].split("_")[0]
    study_id=subj_id[1:4]
    study_id=int(study_id)
    
    df=pd.read_csv(i)

    practice=df['Response.keys'].dropna().tolist()
    
    learn_df = df.dropna(subset=['Answer.keys'])
    learn_list = learn_df[['Answer.keys', 'Answer_2.keys', 'Answer_3.keys']].values.ravel().tolist()

    novel=df['Answer_4.keys'].dropna().tolist()
    noise=df['Answer_5.keys'].dropna().tolist()
    
    
    data = [study_id, 'ra_record_keeping_arm_1', subj_id, df['date'].iloc[0].split("_")[0] ] + practice + learn_list + novel + noise + [2]
        
    cfmt_u_upload.loc[len(cfmt_u_upload)] = data

## RMIE

In [42]:
rmie_files=glob.glob(raw_data_loc  +  "/*RMIE*.csv")

In [44]:
for i in rmie_files:

    subj_id=i.split("/")[-1].split("_")[0]
    
    
    df=pd.read_csv(i)
    date=df['date'].iloc[0].split("_")[0]
    
    
    responses=df['key_resp.keys'].dropna().tolist()
    data=[subj_id, date ] + responses + [2]
    
    
    rmie_upload.loc[len(rmie_upload)] = data

## CFMT Inverted 

In [46]:
cfmt_i_files=glob.glob(raw_data_loc  +  "/*CFMT_I*.csv")

In [60]:
for i in cfmt_i_files:
 
    subj_id=i.split("/")[-1].split("_")[0]
    study_id=subj_id[1:4]
    study_id=int(study_id)
    
    df=pd.read_csv(i)

    practice=df['Response.keys'].dropna().tolist()
    
    learn_df = df.dropna(subset=['Answer.keys'])
    learn_list = learn_df[['Answer.keys', 'Answer_2.keys', 'Answer_3.keys']].values.ravel().tolist()

    novel=df['Answer_4.keys'].dropna().tolist()
    noise=df['Answer_5.keys'].dropna().tolist()
    
    question=df['key_resp_6.keys'].iloc[63]
    
    data = [ subj_id, df['date'].iloc[0].split("_")[0] ] + practice + learn_list + novel + noise + [question] + [2]
        
    cfmt_i_upload.loc[len(cfmt_i_upload)] = data

## Export

In [66]:
merged_df = pd.merge(cfmt_u_upload, rmie_upload, left_on='cfmt_a_u_part_id', right_on='rmie_a_part_id')
merged_df = pd.merge(merged_df, cfmt_i_upload, left_on='cfmt_a_u_part_id', right_on='cfmt_a_i_part_id')

,studyid,redcap_event_name,cfmt_a_u_part_id,cfmt_a_u_date_admin,cfmt_a_u_pa,cfmt_a_u_pb,cfmt_a_u_pc,cfmt_a_u_01,cfmt_a_u_02,cfmt_a_u_03,...,cfmt_a_i_65,cfmt_a_i_66,cfmt_a_i_67,cfmt_a_i_68,cfmt_a_i_69,cfmt_a_i_70,cfmt_a_i_71,cfmt_a_i_72,cfmt_a_i_hard,cambridge_face_and_memory_test_adults_inverted_complete
0,0,ra_record_keeping_arm_1,P000,2024-05-04,1.0,2.0,2.0,2.0,3.0,3.0,...,2.0,1.0,2.0,3.0,2.0,1.0,2.0,3.0,1.0,2
1,0,ra_record_keeping_arm_1,P000,2024-05-04,1.0,2.0,2.0,2.0,3.0,3.0,...,2.0,1.0,2.0,3.0,2.0,1.0,2.0,3.0,1.0,2
2,0,ra_record_keeping_arm_1,P000,2024-05-04,1.0,2.0,2.0,2.0,2.0,3.0,...,2.0,1.0,2.0,3.0,2.0,1.0,2.0,3.0,1.0,2
3,0,ra_record_keeping_arm_1,P000,2024-05-04,1.0,2.0,2.0,2.0,2.0,3.0,...,2.0,1.0,2.0,3.0,2.0,1.0,2.0,3.0,1.0,2


In [69]:
merged_df = merged_df.apply(lambda x: x.astype(int) if x.dtype == np.float64 else x)

In [19]:
for_export=merged_df.set_index('studyid')

In [20]:
from datetime import date
today = date.today()
date = today.strftime("%Y%m%d")

In [21]:
for_export.to_csv(f'CSV_for_RedCap_Upload_{date}.csv')

In [25]:
source = 'data_raw'
destination = 'uploaded_data'

if not os.path.exists(destination):
    os.makedirs(destination)
    

allfiles = os.listdir(source)
 
for file in allfiles:
    src_path = os.path.join(source, file)
    dst_path = os.path.join(destination, file)
    os.rename(src_path, dst_path)